In [ ]:
import rpy2
import scipy
import logging
import warnings
import anndata2ri
import pandas as pd
import scanpy as sc
import numpy as np
import seaborn as sb
import decoupler as dc
import scrublet as scr
import decoupler as dc
from scipy import sparse
from anndata import AnnData
from tabnanny import verbose
import matplotlib.pyplot as plt
from gsva_prep import prep_gsva
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from typing import Optional, Union
from matplotlib.pyplot import rcParams
from functions import pathway_analyses
from statsmodels.stats.multitest import multipletests
from sklearn.model_selection import train_test_split
from pytorch_lightning.loggers import TensorBoardLogger
from rpy2.robjects.conversion import localconverter

In [ ]:
def get_sys_dpi(width, height, diag):
    '''
    obtain dpi of system
    
    w: width in pixels (if unsure, go vist `whatismyscreenresolution.net`)
    h: height in pixels
    d: diagonal in inches
    '''
    w_inches = (diag**2/ (1 + height**2/width**2))**0.5
    return round(width/w_inches)

In [ ]:
# # Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

# # Automatically convert rpy2 outputs to pandas dataframes
# pandas2ri.activate()
# anndata2ri.activate()
# %load_ext rpy2.ipython

warnings.filterwarnings("ignore", category=PendingDeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

rcParams['figure.dpi'] = get_sys_dpi(1512, 982, 14.125)
#rcParams['figure.figsize']=(4,4) #rescale figures

sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
sc.logging.print_versions()



# **hdWGCNA in single-cell data**

Here we use `hdWGCNA` to perform co-expression network analysis on single-cell data. 

`hdWGCNA` is an R package for performing `weighted gene co-expression network analysis (WGCNA)` in high dimensional transcriptomics data such as single-cell RNA-seq or spatial transcriptomics. hdWGCNA is highly modular and can construct co-expression networks across multi-scale cellular and spatial hierarchies. hdWGNCA identifies robust modules of inerconnected genes, and provides context for these modules through various biological knowledge sources. hdWGCNA requires data formatted as Seurat objects, one of the most ubiquitous formats for single-cell data. 

`Note: hdWGCNA is under active development, so you may run into errors and small typos. We welcome users to write GitHub issues to report bugs, ask for help, and to request potential enhancements.`

Explore the full capabilites of hWGNA in the following publications:

- [**Morabito et al. bioRxiv 2022**](https://www.biorxiv.org/content/10.1101/2022.09.22.509094v1)
- [**Morabito & Miyoshi et al. Nature Genetics 2021**](https://www.nature.com/articles/s41588-021-00894-z)


Here, apply hdWGCNA to the processed single-nucleus RNA-seq (snRNA-seq) datasets. These dataset have already been fully processed using a standard single-cell transcritpomics analysis pipelines (please see the following notebooks, `preprocessing_data_integration`, `preprocessing_quality_control`, `preprocessing_cluster_annotation`). 


If you would like to apply this notebook to your own dataset, you first need to satisfy the following prerequisites:

A single-cell or single-nucleus transcriptomics dataset in Seurat format.
Normalize the gene expression matrix NormalizeData.
Identify highly variable genes VariableFeatures.
Scale the normalized expression data ScaleData
Perform dimensionality reduction RunPCA and batch correction if needed RunHarmony.
Non-linear dimensionality reduction RunUMAP for visualizations.
Group cells into clusters (FindNeighbors and FindClusters).
An example of running the prerequisite data processing steps can be found in the Seurat Guided Clustering Tutorial.


In [ ]:
%%R
suppressPackageStartupMessages({
    library(WGCNA)
    library(Matrix)
    library(viridis)
    library(harmony)
    library(ggpubr)
    library(tictoc)
    library(RColorBrewer)
    library(Hmisc)
    library(corrplot)
    library(grid)
    library(gridExtra)
    library(igraph)
    library(ggrepel)
    library(readxl)
    library(conflicted)



    # single-cell analysis package
    library(Seurat)

    # plotting and data science packages
    library(tidyverse)
    library(cowplot)
    library(patchwork)

    # co-expression network analysis packages:
    library(WGCNA)
    library(hdWGCNA)

    # gene enrichment packages
    library(enrichR)
    library(GeneOverlap)


    library(GSEABase)
    library(GSVA) 
# needs to be run every time you start R and want to use %>%
})

# using the cowplot theme for ggplot
theme_set(theme_cowplot())

# set random seed for reproducibility
set.seed(12345)

# optionally enable multithreading
# enableWGCNAThreads(nThreads = 4)

## **Prepare data**

Now, we load the preprocessed and annotated data for downstream analysis.

In [ ]:
save_prefix = 'leng_sfg'

adata_annot = sc.read_h5ad(f'../data/raw/{save_prefix}/{save_prefix}_raw_anndata.h5ad')
adata_annot.obs_names_make_unique()
adata_annot.var_names_make_unique()

try:
    adata_annot.X = adata_annot.layers['counts'].copy()
except KeyError:
    print('"counts" not in layers...')
    print('analysis requires unnormalized count data...')
    print('analysis proceesing with data in "adata.x"...')
    
del adata_annot.obsm, adata_annot.layers, adata_annot.varm, adata_annot.uns, adata_annot.obsp

print(adata_annot)

In [ ]:
get_cell_types = True # whether to  to obtain appropriate cell-type columns: True/False
map_meta = True
deg_method =  'DESeq2-Wald'
test_names = ['late_vs_early', 'early_vs_no', 'late_vs_no', 'ad_vs_no']
filter_genes = "TRUE"
subject_ids_for_study = {'leng_sfg': 'PatientID',
                        'leng_etc': 'PatientID',
                        'seaad_mtg': 'individualID'}

subject_id = subject_ids_for_study[save_prefix]     # for leng this is `PatientID` for mathys is 'Subject', and allen is 'individualID'
gene_celltype_threshold = 0.10          # determines number of cells the gene must be expressed in 
covariates = ['None']                   # list of covariates to be accounted for in regression.
gene_selection = 'custom'               # specifies the gene selection method when setting up seurat object for WGCNA. The th
celltypes = ["Excitatory", "Inhibitory", "Astrocyte", "Microglia", "Oligodendrocyte", "OPC"]
metadata = f'../data/raw/{save_prefix}/{save_prefix}_metadata.csv' 
meta = pd.read_csv(metadata, encoding_errors='ignore')

gene_selection = 'custom'   # specifies the gene selection method when setting up seurat object for WGCNA. The th
                            # Posible values are "custom", "fraction", "variable"
                            # If custom, a list of genes must be passed.

gene_set_select = 'diff_exp' # If gene_selection = 'custom'. This specifies how to obtain the list of
                             # genes to pass into `SetupForWGCNA`. # The posible values are "diff_exp", "overlap", "all"


In [ ]:

mapping = {'leng_etc':
           
           {'Exc': 'Excitatory', 
            'Inh': 'Inhibitory', 
            'Astro': 'Astrocyte',
            'Endo': 'Endothelial', 
            'Micro': 'Microglia', 
            'OPC': 'OPC', 
            'Oligo': 'Oligodendrocyte'},

           'leng_sfg':
           
           {'Exc': 'Excitatory', 
            'Inh': 'Inhibitory', 
            'Astro': 'Astrocyte',
            'Endo': 'Endothelial', 
            'Micro': 'Microglia', 
            'OPC': 'OPC', 
            'Oligo': 'Oligodendrocyte'},
            
            'seaad_mtg':
           
           {'Excitatory': 'Excitatory', 
            'Inhibitory': 'Inhibitory',
            'Astrocyte': 'Astrocyte',
            'Microglia': 'Microglia', 
            'Endothelial': 'Endothelial', 
            'OPC': 'OPC',
            'Oligodendrocyte': 'Oligodendrocyte'},
            }

cell_column = {'leng_etc': 'clusterCellType',
               'leng_sfg': 'clusterCellType',
               'seaad_mtg': 'cell_type',
               }

In [ ]:
if get_cell_types:
    adata_annot.obs['cell_type'] = adata_annot.obs[cell_column[save_prefix]].map(mapping[save_prefix])

In [ ]:
# map the pathology group to teh subject id in .obs
adata_annot = adata_annot[adata_annot.obs.cell_type.isin(celltypes)]
adata_annot.obs['pathology.group'] = adata_annot.obs[subject_id].map(dict(zip(meta[subject_id].astype(str), meta['pathology.group'])))

In [ ]:
%%R -i adata_annot -i subject_id -i gene_celltype_threshold -i celltypes -i test_names -i save_prefix -i gene_selection

print(adata_annot)
print('loaded data into memory for recursive use')

In [ ]:
del adata_annot

## **Set up Seurat object for WGCNA**

Before running hdWGCNA, we first have to set up the Seurat object. Most of the information computed by hdWGCNA is stored in the Seurat object’s `@misc slot`. 

The `SetupForWGCNA` function selects the genes that will be used for WGCNA using three different approaches using the `gene_select` parameter:

- `variable`: use the genes stored in the Seurat object’s `VariableFeatures`.

- `fraction`: use genes that are expressed in a certain fraction of cells for in the whole dataset or in each group of cells, specified by group.by.

- `custom`: use genes that are specified in a custom list.

For our specific case, the `gene_selection` parameter is set to `custom`, we employ the genes present in all the overlapping pathways in a cell type-specific and study-specific manner.

To accomplish this, we load the list of overlapping pathways that was previously saved in `../results/{test_name}/{test_name}_pathway_overlap.csv` from the `pathway_meta_analysis.ipynb` notebook. Additionally, we load the genes from the `.gmt` file saved in `../results/gsva/{study}/{cur_celltype}/gs.gmt` from the `pathway_analysis.ipynb` notebook.

##### **Select Gene Set**

To obtain the gene sets required for the `SetupForWGCNA` function, we consider the `gene_set_select` variable when `gene_select` is set to `custom`. The `gene_set_select` variable determines how the `gene_list` passed into the `SetupForWGCNA` function will be obtained. 

There are three options available:

- `diff_exp`: This option utilizes the genes present in the differentially expressed pathways, previously computed and saved to the `../results/{test_name}/{save_prefix}_differentially_expressed_pathways.csv` file.

- `overlap`: Here, the genes used are those expressed in pathways that overlap across different studies. The pathway overlap information is retrieved from the `../results/{test_name}/{test_name}_pathway_overlap.csv` file.

- `all`: With this option, all genes in the pathways stored in the `../results/gsva/{save_prefix}/{celltype}/gs.gmt` file are included.

Here, we specifically set `gene_select=custom` and `gene_set_select = diff_exp`. However, you can customize the gene selection process for your hdWGCNA experiment.


In [ ]:
%%R -o geneSets -i gene_set_select

# iterate over celltypes and obtain 
geneSets <- list()
for(test_name in test_names){

  geneSets[[test_name]] <- list()
  for (cur_celltype in celltypes){
    # get gene sets
    geneSets[[test_name]][[cur_celltype]] <- getGmt(file.path('../results/gsva/', save_prefix, '/', cur_celltype, 'gs.gmt'))
    geneSets[[test_name]][[cur_celltype]] <- geneIds(geneSets[[test_name]][[cur_celltype]])

    if (gene_set_select=='overlap'){
      pathways <- read.csv(paste0('../results/',test_name, '/', test_name, '_pathway_overlap.csv'))
      pathways <- pathways$pathway
    }else if (gene_set_select=='diff_exp'){
      pathways <- read.csv(paste0('../results/', test_name, '/', save_prefix, '/Data/differentially_expressed_pathways.csv'))
      pathways <- subset(pathways, P.Value < 0.05 & celltype == cur_celltype)$pathway
    }else if (gene_set_select=='all'){
      pathways <- names(geneSets[[test_name]][[cur_celltype]])
    }

    geneSets[[test_name]][[cur_celltype]] <-  geneSets[[test_name]][[cur_celltype]][names(geneSets[[test_name]][[cur_celltype]]) %in% pathways]
    geneSets[[test_name]][[cur_celltype]] <- unique(unlist(geneSets[[test_name]][[cur_celltype]], recursive = FALSE))

    print(length(geneSets[[test_name]][[cur_celltype]]))
  }
}

In [ ]:
# convert nested list of geneSets object into Rpy2 object 

geneSets =  robjects.ListVector(
                        {
                            test_name: robjects.ListVector(
                                    {cell_type: geneSets[test_name][cell_type]
                                                 
                                    for cell_type in celltypes
                                    }
                                )
                            for test_name in test_names
                        }

                        )

In [ ]:
%%R -i geneSets

seurat_obj <- as.Seurat(adata_annot, counts = "X", data = "X")

print(seurat_obj)

# Perform dimensionality reduction and plot

seurat_obj <- FindVariableFeatures(seurat_obj)
seurat_obj <- ScaleData(seurat_obj)
seurat_obj <- RunPCA(seurat_obj)
seurat_obj <- RunHarmony(seurat_obj, group.by.vars = subject_id)
seurat_obj <- RunUMAP(seurat_obj, reduction='harmony', n.neighbors=15, dims=1:30, min.dist=0.1)

p <- DimPlot(seurat_obj, group.by = "cell_type", label = TRUE) +
     umap_theme() + ggtitle("leng") + NoLegend()

fig_dir = paste0("../results/hdWGCNA/UMAP/")

if (!dir.exists(fig_dir)) {
  dir.create(fig_dir, recursive=TRUE)
}

# Save plot to PDF
pdf(file = paste0(fig_dir, save_prefix, "_integrated_umap.pdf"), width = 4, height = 4, useDingbats = FALSE)
print(p)
dev.off()

# create a hdWGCNA experiment for each celltype
seurat_dat <- seurat_obj
seurat_obj <- list()


for (cur_cell_type in celltypes){

  seurat_obj[[cur_cell_type]] <- subset(seurat_dat, cell_type == cur_cell_type)

  print(paste0('Creating hdWGNA Experiment for ', toupper(cur_cell_type)))

  if (gene_selection == 'custom') {
    seurat_obj[[cur_cell_type]] <- SetupForWGCNA(
      seurat_obj[[cur_cell_type]],
      gene_select = "custom",                                            # the gene selection approach
      gene_list = as.vector(geneSets[['ad_vs_no']][[cur_cell_type]]),    # list of genes to be included
      group.by = 'cell_type',                                            # grouping parameter
      wgcna_name = toupper(substr(cur_cell_type, 1, 3))                  # the name of the hdWGCNA experiment
    )
  } else {
    seurat_obj[[cur_cell_type]] <- SetupForWGCNA(
      seurat_obj[[cur_cell_type]],
      gene_select = "fraction",                               # the gene selection approach
      fraction = gene_celltype_threshold,                     # fraction of cells for gene inclusion
      group.by = 'cell_type',                                 # grouping parameter
      wgcna_name = toupper(substr(cur_cell_type, 1, 3))       # the name of the hdWGCNA experiment
    )
  }

  print(seurat_obj[[cur_cell_type]])
  print(paste0(length(GetWGCNAGenes(seurat_obj[[cur_cell_type]])), " WGCNA Genes"))

}

rm(seurat_dat)
rm(adata_annot)

## **Construct metacells**

In the hdWGCNA pipeline, the initial step involves constructing metacells from a single-cell dataset using the `MetacellsByGroups` function, which aggregates similar cells using the `k-Nearest Neighbors` algorithm. This process is essential for reducing the sparsity of the gene expression matrix and involves grouping by specific parameters like `Sample` and `cell_type` to ensure metacells originate from the same biological sample. The function allows tuning of the number of cells aggregated (k) and the maximum shared overlap, with a min_cells parameter to exclude small groups, thereby optimizing the construction of a metacell expression matrix for subsequent analysis.

In [ ]:
%%R

# construct metacells in each group for each wgnca experiment

for (cell_type in celltypes){

  print(paste0('Constructing MetaCells in hdWGCNA Experiment for ', toupper(cell_type)))
  
  seurat_obj[[cell_type]] <- MetacellsByGroups(
      
    seurat_obj = seurat_obj[[cell_type]],
    group.by = c("cell_type", subject_id), # specify the columns in seurat_obj@meta.data to group by
    reduction = 'harmony',  # select the dimensionality reduction to perform KNN on
    k = 25, # nearest-neighbors parameter
    max_shared = 10, # maximum number of shared cells between two metacells
    ident.group = 'cell_type', # set the Idents of the metacell seurat object
    wgcna_name = toupper(substr(cell_type, 1, 3)),        # the name of the hdWGCNA experiment
    )

    # normalize metacell expression matrix:
  seurat_obj[[cell_type]] <- NormalizeMetacells(seurat_obj[[cell_type]],  wgcna_name = toupper(substr(cell_type, 1, 3)))
}


## **Co-expression network analysis**

### **Set up the expression matrix**

the `SetDatExpr` function is used to set up the expression matrix for network analysis, involving subsetting the data to include only a specific cell type per run. By default, the metacell expression matrix is utilized `(use_metacells=TRUE)`, though the option to use the single-cell expression matrix is available. This function also allows users to choose the source slot for the expression matrix, accommodating different normalization methods like SCTransform or NormalizeData.

In [ ]:
%%R

for (cell_type in celltypes){

    print(paste0('Setting up expression matrix in hdWGCNA Experiment for ', toupper(cell_type)))

    seurat_obj[[cell_type]] <- SetDatExpr(
      seurat_obj[[cell_type]],
      group_name = cell_type, # the name of the group of interest in the group.by column
      group.by = 'cell_type', # the metadata column containing the cell type info. This same column should have also been used in MetacellsByGroups
      assay = 'originalexp', # using RNA assay
      slot = 'data', # using normalized data
      wgcna_name = toupper(substr(cell_type, 1, 3)) # the name of the hdWGCNA experiment
    )
}

### **Select soft-power threshold**

Next we will select the “soft power threshold”. Selecting the `soft power threshold` is a crucial step, as it influences the construction of the gene-gene correlation adjacency matrix used to infer co-expression relationships between genes. The `TestSoftPowers` function is employed to conduct a parameter sweep for various soft power thresholds, aiding in choosing an appropriate value to ensure the co-expression network approximates a scale-free topology. Additionally, the `PlotSoftPowers`` function is used to visually represent the outcomes of this parameter sweep, to assess the network's structure at different thresholds.

In [ ]:
%%R

fig_dir = paste0("../results/hdWGCNA/SoftPower/", save_prefix, '/')

if (!dir.exists(fig_dir)) {
  dir.create(fig_dir, recursive=TRUE)
}


for (cell_type in celltypes){
  
  print(paste0('Estinmating Soft-Power Threshold in hdWGCNA Experiment for ', toupper(cell_type)))

  # Test different soft powers:
  seurat_obj[[cell_type]] <- TestSoftPowers(
    seurat_obj[[cell_type]],
    networkType = 'signed', # you can also use "unsigned" or "signed hybrid"
    setDatExpr = TRUE,
    group.by = 'cell_type',
    group_name = cell_type,
  )

  # plot the results:
  plot_list <- PlotSoftPowers(seurat_obj[[cell_type]], 
                              # wgcna_name = toupper(substr(cell_type, 1, 3)) # the name of the hdWGCNA experiment
                              )

  # assemble with patchwork
  pdf(paste0(fig_dir, cell_type, '_SoftPower.pdf'), width=12, height=8)
  print(wrap_plots(plot_list, ncol=2))
  dev.off()
}



For WGCNA and hdWGCNA, the recommended approach is to choose the lowest soft power threshold that achieves a Scale Free Topology Model Fit of 0.8 or higher. If not manually specified, the ConstructNetwork function will automatically select an appropriate soft power threshold during the network construction process.

### **Construct co-expression network**

We now have everything that we need to construct our co-expression network. Here we use the hdWGCNA function `ConstructNetwork`, which internally utilizes the `WGCNA` function `blockwiseConsensusModules`. While this function offers numerous parameters for advanced customization, default settings are generally effective for many single-cell datasets. Parameters for blockwiseConsensusModules can be directly passed to ConstructNetwork using the same names, allowing for the construction of the co-expression network based on the previously selected soft power threshold.

In [ ]:
%%R

  
tom_outdir <-  paste0('../results/hdWGCNA/TOM/', save_prefix, '/')

# make output dir for the TOM
if(!dir.exists(tom_outdir)){
  dir.create(tom_outdir, recursive=TRUE)
}

for (cell_type in celltypes) {

  print(paste0('Constructing co-expression network in hdWGCNA Experiment for ', toupper(cell_type)))
  
  tryCatch({
    # construct co-expression network:
    seurat_obj[[cell_type]] <- ConstructNetwork(
      seurat_obj[[cell_type]], 
      soft_power = NULL, # Set to NULL so that the ConstructNetwork function obtains it automatically (Scale Free Topology Model Fit >= 0.8)
      setDatExpr = FALSE,
      overwrite_tom = TRUE,
      tom_outdir = tom_outdir,
      tom_name = toupper(substr(cell_type, 1, 3)) # name of the topological overlap matrix written to disk set as the name of the hdWGCNA experiment 
    )
  }, error = function(e) {
    print(paste0('Error encountered while processing ', toupper(cell_type)))
    print(paste0(toupper(cell_type), ' dropped from experiment'))
    # Code to handle the error condition
    seurat_obj <- seurat_obj[names(seurat_obj) != cell_type]
    assign("seurat_obj", seurat_obj, envir = .GlobalEnv) # Update the modified seurat_obj in the global environment

  },  
  message = function(m) {
    print(paste0('Error encountered while processing ', toupper(cell_type)))
    print(paste0(toupper(cell_type), ' dropped from experiment'))
    # Code to handle the error condition
    seurat_obj <- seurat_obj[names(seurat_obj) != cell_type]
    assign("seurat_obj", seurat_obj, envir = .GlobalEnv) # Update the modified seurat_obj in the global environment

  }) 
  
}


print(names(seurat_obj))


hdWGCNA also includes a function PlotDendrogram function for visualizing the WGCNA dendrogram, a standard method to display the various co-expression modules derived from network analysis. In this dendrogram, each leaf represents a gene, with colors indicating their module assignments. `Notably, genes in the "grey" module, which were not grouped into any specific co-expression module, should be excluded from all subsequent analyses and interpretations.`

In [ ]:
%%R

fig_dir = paste0("../results/hdWGCNA/dendrogram/", save_prefix, '/')

if (!dir.exists(fig_dir)) {
  dir.create(fig_dir, recursive=TRUE)
}

tryCatch({
    for (cell_type in names(seurat_obj)){
        # print(paste0('Constructing co-expression networkd in hdWGCNA Experiment for ', toupper(cell_type)))
        pdf(paste0(fig_dir, cell_type, "_dendro.pdf"), height=2, width=4)
        PlotDendrogram(seurat_obj[[cell_type]], main='hdWGCNA Dendrogram')
        dev.off()
    }
}, error = function(e){
    NULL
})

### **Module Eigengenes and Connectivity**

### Compute harmonized module eigengenes

`Module Eigengenes (MEs)` summarize the gene expression profile of co-expression modules in hdWGCNA, calculated by performing principal component analysis (PCA) on each module's subset of the gene expression matrix, with the first principal component as the ME. Given that single-cell data is sensitive to technical artifacts, hdWGCNA's ModuleEigengenes function not only computes MEs but also offers Harmony batch correction to produce `harmonized module eigengenes (hMEs)`. This harmonization is executed with a focus on the sample of origin, as specified by the `group.by.vars` parameter.

In [ ]:
%%R

for (cell_type in names(seurat_obj)){

    modules <- GetModules(seurat_obj[[cell_type]])

    if (!(length(unique(modules$module))<=2 &  'grey'%in%modules$module)){

        print(paste0('Estimating module eigen-genes in hdWGCNA Experiment for ', toupper(cell_type)))
        
        # fneed to run ScaleData first or else harmony throws an error:
        seurat_obj[[cell_type]] <- ScaleData(seurat_obj[[cell_type]], features=VariableFeatures(seurat_obj[[cell_type]]))

        # compute all MEs in the full single-cell dataset
        seurat_obj[[cell_type]] <- ModuleEigengenes(
        seurat_obj[[cell_type]],
        group.by.vars=subject_id
        )
        print('----------------')

    }   else{    

        print(paste0('Error encountered while processing ', toupper(cell_type)))
        print(paste0(toupper(cell_type), ' dropped from experiment'))
        # Code to handle the error condition
        seurat_obj <- seurat_obj[names(seurat_obj) != cell_type]
        assign("seurat_obj", seurat_obj, envir = .GlobalEnv) # Update the modified seurat_obj in the global environment

    } 
    
}

The ME matrices are stored as a matrix where each row is a cell and each column is a module. This matrix can be extracted from the Seurat object using the GetMEs function, which retrieves the hMEs by default.

In [ ]:
%%R -o hMEs -o MEs

hMEs <- list()
MEs <- list()

for (cell_type in names(seurat_obj)){
    print(paste0('Getting harmonized and non-harmonized MEs in hdWGCNA Experiment for ', toupper(cell_type)))
    
    # harmonized module eigengenes:
    hMEs[[cell_type]] <- GetMEs(seurat_obj[[cell_type]])

    # module eigengenes:
    MEs[[cell_type]] <- GetMEs(seurat_obj[[cell_type]], harmonized=FALSE)
}

### **Compute module connectivity**

We want to focus on `hub genes`, which are highly connected within each module. To assess this, hdWGCNA includes the `ModuleConnectivity` function to calculate the `eigengene-based connectivity (kME)` of each gene in the full single-cell data. This function computes pairwise correlations between genes and module eigengenes. While kME can be calculated for all cells, it is recommended to compute it specifically in the cell type or group initially used in the ConstructNetwork function.

In [ ]:
%%R

for (cell_type in names(seurat_obj)){

  print(paste0('Estimating module connectivity in hdWGCNA Experiment for ', toupper(cell_type)))

  # compute eigengene-based connectivity (kME):
  seurat_obj[[cell_type]] <- ModuleConnectivity(
    seurat_obj[[cell_type]],
    group.by = 'cell_type', 
    group_name = cell_type
  )
  
}

For convenience, we re-name the hdWGCNA modules to indicate that they are from a cell-type group.

In [ ]:
%%R

for (cell_type in names(seurat_obj)){

  print(paste0('Renaming modules in hdWGCNA Experiment for ', toupper(cell_type)))

  # rename the modules
  seurat_obj[[cell_type]] <- ResetModuleNames(
    seurat_obj[[cell_type]],
    new_name = paste0(toupper(substr(cell_type, 1, 3)), "-M"),
    wgcna_name = toupper(substr(cell_type, 1, 3)) # the name of the hdWGCNA experiment,
  )
}

We also reset module colors

In [ ]:
%%R


library(MetBrewer)

for (cell_type in names(seurat_obj)){

  print(paste0('Resetting module colors in hdWGCNA Experiment for ', toupper(cell_type)))


  modules <- GetModules(seurat_obj[[cell_type]])
  mods <- levels(modules$module)
  mod_colors <- dplyr::select(modules, c(module, color)) %>%
    distinct %>% arrange(module) %>% .$color
  n_colors <- length(mod_colors) -1

  new_colors <- paste0(met.brewer("Signac", n=n_colors,))
  new_colors <- sample(new_colors)
  seurat_obj[[cell_type]] <- ResetModuleColors(seurat_obj[[cell_type]], new_colors)

}


We can visualize the genes in each module ranked by kME using the PlotKMEs function.

In [ ]:
%%R

library(conflicted)
conflicts_prefer(dplyr::select)

fig_dir = paste0("../results/hdWGCNA/ModuleConnectivity/", save_prefix, '/')

if (!dir.exists(fig_dir)) {
  dir.create(fig_dir, recursive=TRUE)
}

for (cell_type in names(seurat_obj)){
    
    print(paste0('Visualize Module Connectivity (kMEs) in hdWGCNA Experiment for ', toupper(cell_type)))

    # plot genes ranked by kME for each module

    p <- PlotKMEs(seurat_obj[[cell_type]], ncol=5)

    pdf(paste0(fig_dir, cell_type, '_KMEs.pdf'), width=12, height=8)
    print(p)
    dev.off()
}

### Getting the module assignment table

The hdWGCNA package provides the `GetModules` function for convenient access to the module assignment table. This table includes three primary columns: gene_name for the gene symbol or ID, module for the gene's module assignment, and color for the color mapping of each module, which is frequently used in downstream plotting. Additionally, if ModuleConnectivity has been used in the hdWGCNA experiment, the table will feature extra columns showing the kME values for each module.

In [ ]:
%%R

for (cell_type in names(seurat_obj)){
    
    print(paste0('Obtain Module Assignment Table in hdWGCNA Experiment for ', toupper(cell_type)))

    # get the module assignment table:
    modules <- GetModules(seurat_obj[[cell_type]])

    # show the first 6 columns:
    print(head(modules[,1:6]))
}

A table of the top N hub genes sorted by kME can be extracted using the GetHubGenes function.

In [ ]:
%%R

for (cell_type in names(seurat_obj)){
    
    print(paste0('Obtain Hub Genes in hdWGCNA Experiment for ', toupper(cell_type)))

    # get hub genes
    hub_df <- GetHubGenes(seurat_obj[[cell_type]], n_hubs = 10)

    print(head(hub_df))
}

## **Compute hub gene signature scores**

Gene scoring analysis is a popular method in single-cell transcriptomics for computing a score for the overall signature of a set of genes.

hdWGCNA offers the `ModuleExprScore` function, which computes gene scores for a specified number of genes in each module, employing either the Seurat or UCell algorithm. This gene scoring method serves as an alternative to summarizing module expression, differing from the approach of computing module eigengenes.

In [ ]:
%%R 

for (cell_type in names(seurat_obj)){
    
  print(paste0('Compute hub gene scores in hdWGCNA Experiment for ', toupper(cell_type)))
    
  # compute gene scoring for the top 25 hub genes by kME for each module
  # with Seurat method
  # seurat_obj[[cell_type]] <- ModuleExprScore(
  #   seurat_obj[[cell_type]],
  #   n_genes = 25,
  #   method='Seurat'
  # )

  # compute gene scoring for the top 25 hub genes by kME for each module
  # with UCell method
  library(UCell)
  seurat_obj[[cell_type]] <- ModuleExprScore(
    seurat_obj[[cell_type]],
    n_genes = 25,
    method='UCell'
  )
}

# **Save Seurat Object**

This wraps up the critical analysis steps for hdWGCNA, let's save our output.

In [ ]:
%%R -i seurat_obj

dat_dir = paste0("../results/hdWGCNA/SeuratObject/", save_prefix, '/')

if (!dir.exists(dat_dir)) {
  dir.create(dat_dir, recursive=TRUE)
}

for (cell_type in names(seurat_obj)){
    
    print(paste0('Saving hdWGCNA object in hdWGCNA Experiment for ', toupper(cell_type)))

    saveRDS(seurat_obj[[cell_type]], file=paste0(dat_dir, cell_type, '_hdWGCNA_object.rds'))

}